In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json

# Scraping Notebook
This notebook is used to scrape and compile all data needed for the project. First, it will scrape the top 15 games of each genre from steampowered.com with their game id's, then use those ids to gather review data from SteamWorks API

In [270]:
# Gathering 15 games and id numbers from each genre

#each url is formatted the same for the top rated games, so creating a list of the genres to cycle through
genre_list = ['Action', 'Adventure', 'Casual', 'Indie', 'Massively%20Multiplayer',
             'Racing', 'RPG','Simulation', 'Sports', 'Strategy' ]


games = {}
#cycle through each genre, pull 15 games
for genre in genre_list:
#     response = requests.get('https://store.steampowered.com/tags/en/{}/#p=0&tab=TopRated'.format(genre))
#     page = response.text
#     soup = BeautifulSoup(page, 'html5lib')
#     table = soup.find('div', id = 'TopRatedRows')
#     rows = [row for row in table.find_all('a')]
    
#     #for each genre, get the 15 titles and ids
#     for row in rows:
#         title = (row.find(class_='tab_item_name').text)
#         app_id = row.get('href').split('/')[4]
    
#         games[title] = (genre, app_id)
        
    # adding second page for each genre, making it 30 games per genre
    
    response = requests.get('https://store.steampowered.com/tags/en/{}/#p=1&tab=TopRated'.format(genre))
    page = response.text
    soup = BeautifulSoup(page, 'html5lib')
    table = soup.find('div', id = 'TopRatedRows')
    rows = [row for row in table.find_all('a')]
    
    #for each genre, get the 15 titles and ids
    for row in rows:
        title = (row.find(class_='tab_item_name').text)
        app_id = row.get('href').split('/')[4]
    
        games[title] = (genre, app_id)
    
    

In [ ]:
for genre in genre_list:
    response = requests.get('https://store.steampowered.com/tags/en/{}/#p=1&tab=TopRated'.format(genre))
    page = response.text
    soup = BeautifulSoup(page, 'html5lib')
    table = soup.find('div', id = 'TopRatedRows')
    rows = [row for row in table.find_all('a')]
    
    #for each genre, get the 15 titles and ids
    for row in rows:
        title = (row.find(class_='tab_item_name').text)
        app_id = row.get('href').split('/')[4]
    
        games[title] = (genre, app_id)

In [271]:
len(games)

125

In [267]:
#games is a dictionary, with game titles as keys, and rpg and gameid as values

#games

In [272]:
df = pd.DataFrame(games).T

In [273]:
df

,0,1
Counter-Strike: Global Offensive,Action,730
Grand Theft Auto V,Action,271590
Tom Clancy's Rainbow Six® Siege,Action,359550
Left 4 Dead 2,Action,550
Warframe,Action,230410
...,...,...
Total War: WARHAMMER II,Strategy,594570
Hearts of Iron IV,Strategy,394360
shapez.io,Strategy,1318690
Slay the Spire,Strategy,646570


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 125 entries, Counter-Strike: Global Offensive to Slay the Spire
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       125 non-null    object
 1   1       125 non-null    object
dtypes: object(2)
memory usage: 2.9+ KB


In [46]:
df[0].value_counts()

Sports                     15
Strategy                   15
Simulation                 15
Massively%20Multiplayer    13
RPG                        12
Adventure                  12
Action                     12
Casual                     11
Indie                      10
Racing                     10
Name: 0, dtype: int64

In [48]:
df[df[0] == "Indie"]

,0,1
Don't Starve,Indie,219740
The Binding of Isaac: Rebirth,Indie,250900
Hollow Knight,Indie,367520
Dead Cells,Indie,588650
FTL: Faster Than Light,Indie,212680
Beat Saber,Indie,620980
Grimm,Indie,252150
Helltaker,Indie,1289310
To the Moon,Indie,206440
The Binding of Isaac,Indie,113200


# Pulling reviews for each game
each api pull retrieves 20 commens, but adding the cursor from the first pull, it will take the next 20. This will get 40 reviews per game.

In [73]:
def get_reviews(idnum):
    reviews=[]
    response = requests.get('http://store.steampowered.com/appreviews/{}?json=1'.format(idnum))
    temp=response.json()
    cursor = temp['cursor']
    for i in range(len(temp['reviews'])):
        reviews.append(temp['reviews'][i]['review'])
    response = requests.get('http://store.steampowered.com/appreviews/{}?json=1{}'.format(idnum, cursor))
    temp = response.json()
    for i in range(len(temp['reviews'])):
        reviews.append(temp['reviews'][i]['review'])
    return reviews

In [74]:
df['idnum'] = df[1]

In [77]:
df.idnum = df.idnum.apply(get_reviews)

In [79]:
df.rename(columns = {'idnum':'reviews'}, inplace=True)

In [99]:
df.head(1)

,0,1,reviews
Counter-Strike: Global Offensive,Action,730,[Your team in every random competitive game:\n...


In [85]:
df.iloc[0]['reviews'][0]

'Your team in every random competitive game:\n\n- You\n- Russian Guy\n- Another Russian Guy\n- A 6 year old kid who slept with your mother\n- Russian Guy who speaks English\n\n10/10'

In [233]:
rando = df.iloc[0]['reviews'][0]

In [234]:
rando in df

False

In [235]:
rando in df.reviews

False

# Trying to seperate list of reviews but keep game name

In [119]:
for i in range(len(df)):
    if rando in df.iloc[i]['reviews']:
        print('true')
#     else:
#         print('not true')

In [101]:
rando in list(df.iloc[0]['reviews'])

False

In [159]:
# df = df.reset_index()
# df.rename(columns={'index':'title'}, inplace=True)
df.rename(columns={0:'genre',1:'appid'},inplace=True)

In [160]:
df.head()

,title,genre,appid,reviews
0,Counter-Strike: Global Offensive,Action,730,[Your team in every random competitive game:\n...
1,Grand Theft Auto V,Action,271590,"[Trevor Philips Enterprises 🥵, After Ep*c Game..."
2,Tom Clancy's Rainbow Six® Siege,Action,359550,[> Play as French guy with giant shield\n> Sto...
3,Left 4 Dead 2,Action,550,"[h, why did they leave me for dead for thats n..."
4,Warframe,Action,230410,"[look at my hours, look at my hours]"


In [109]:
# megadict={}

# def megadicter(list):
#     for items in list:
#         megadict[df.title]=(items)

In [229]:
#df.reviews.apply(megadicter)

In [231]:
# for rows in df:
#     megadict[df.title]=(df.reviews)

In [123]:
megalist=[]

def megalister(list):
    for items in list:
        megalist.append(items)

In [215]:
df.head(1)

,title,genre,appid,reviews,reviews2
0,Counter-Strike: Global Offensive,Action,730,[Your team in every random competitive game:\n...,730


In [217]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     125 non-null    object
 1   genre     125 non-null    object
 2   appid     125 non-null    object
 3   reviews   125 non-null    object
 4   reviews2  125 non-null    object
dtypes: object(5)
memory usage: 5.0+ KB


In [125]:
df.reviews.apply(megalister)

0      None
1      None
2      None
3      None
4      None
       ... 
120    None
121    None
122    None
123    None
124    None
Name: reviews, Length: 125, dtype: object

In [228]:
# list of 4k reviews, but no game title attached. 

#megalist

In [164]:
tester = megalist[0]
tester

'Your team in every random competitive game:\n\n- You\n- Russian Guy\n- Another Russian Guy\n- A 6 year old kid who slept with your mother\n- Russian Guy who speaks English\n\n10/10'

In [134]:
len(df.reviews)

125

In [139]:
tester in df.reviews[124]

False

In [161]:
# this allows me to pull which row from game dataframe the review is in

for i in range(len(df)-1):
    if tester in df.reviews[i]:
        print(df.iloc[i].title)

Counter-Strike: Global Offensive


In [148]:
megalist.to_csv(r'users/michaelharnett/desktop/metis/projects/steam2_redux/megalist.csv')

AttributeError: 'list' object has no attribute 'to_csv'

In [150]:
df.head()

,title,0,1,reviews
0,Counter-Strike: Global Offensive,Action,730,[Your team in every random competitive game:\n...
1,Grand Theft Auto V,Action,271590,"[Trevor Philips Enterprises 🥵, After Ep*c Game..."
2,Tom Clancy's Rainbow Six® Siege,Action,359550,[> Play as French guy with giant shield\n> Sto...
3,Left 4 Dead 2,Action,550,"[h, why did they leave me for dead for thats n..."
4,Warframe,Action,230410,"[look at my hours, look at my hours]"


In [254]:
len(df.iloc[0]['reviews'])

40

In [156]:
df.to_csv(r'/users/michaelharnett/desktop/metis/projects/steam2_redux/megalist.csv', index=False)

# oh wait, there's zip!
should be able to zip each review to the title and genre when creating the list, making a larger data frame wher each review is its own thing

In [162]:
df['reviews2']= df.appid

In [163]:
df.head()

,title,genre,appid,reviews,reviews2
0,Counter-Strike: Global Offensive,Action,730,[Your team in every random competitive game:\n...,730
1,Grand Theft Auto V,Action,271590,"[Trevor Philips Enterprises 🥵, After Ep*c Game...",271590
2,Tom Clancy's Rainbow Six® Siege,Action,359550,[> Play as French guy with giant shield\n> Sto...,359550
3,Left 4 Dead 2,Action,550,"[h, why did they leave me for dead for thats n...",550
4,Warframe,Action,230410,"[look at my hours, look at my hours]",230410


In [174]:
test_dict = {}

def get_reviews_new(idnum):
    reviews=[]
    response = requests.get('http://store.steampowered.com/appreviews/{}?json=1'.format(idnum))
    temp=response.json()
    cursor = temp['cursor']
    for i in range(len(temp['reviews'])):
        #games_dict=dict(zip(idnum,(temp['reviews'][i]['review'])))
        reviews.append(temp['reviews'][i]['review'])
    response = requests.get('http://store.steampowered.com/appreviews/{}?json=1{}'.format(idnum, cursor))
    temp = response.json()
    for i in range(len(temp['reviews'])):
        games_dict=dict(zip(idnum,(temp['reviews'][i]['review'])))             
        #reviews.append(temp['reviews'][i]['review'])
    return games_dict

In [180]:
idlist[0]

'730'

In [186]:
df.reviews2.apply(get_reviews_new)

0                         {'7': 'H', '3': 'e', '0': 'l'}
1      {'2': 'v', '7': 'e', '1': 'r', '5': 'y', '9': ...
2               {'3': 'I', '5': 'e', '9': 'u', '0': 'd'}
3                                   {'5': 'r', '0': 'e'}
4      {'2': 'l', '3': 'o', '0': 'a', '4': 'k', '1': ...
                             ...                        
120    {'4': 'E', '2': 'l', '7': 'c', '5': 'e', '0': ...
121    {'5': 'v', '9': 'k', '4': 'a', '7': 'e', '0': ...
122    {'3': 'e', '9': 'f', '4': 't', '6': 'r', '0': ...
123    {'3': 'W', '2': 'h', '4': 'a', '7': 't', '0': ...
124    {'6': 'i', '4': 'h', '5': 's', '7': ' ', '0': ...
Name: reviews2, Length: 125, dtype: object

In [187]:
games_dict

NameError: name 'games_dict' is not defined

In [189]:
get_reviews_new(idlist[0])

{'7': 'H', '3': 'e', '0': 'l'}

In [236]:
#get_reviews(idlist[0])

In [191]:
df2 = pd.DataFrame()

In [192]:
df2

""


In [195]:
df.explode(df.reviews.str.split(','))

ValueError: column must be a scalar

In [211]:
df.reviews.explode()

0      Your team in every random competitive game:\n\...
0                                Pay $15 to unlock level
0                 counter-fart: globally offensive scent
0      This game saved my life.\n\nI am 27.\n\nMy ex-...
0      Absolutely garbage servers and net-code for pu...
                             ...                        
124                                Damnn this game 10/10
124    10/10 a very fun gameplay, so many different b...
124    Easily the best and most diverse card builder ...
124    A phenomenal card game. Probably the best roug...
124    This game is an easy way to blow through an en...
Name: reviews, Length: 4120, dtype: object

In [258]:
## working code
explodeddf = df.explode('reviews')

In [259]:
explodeddf.to_csv(r'/users/michaelharnett/desktop/metis/projects/steam2_redux/data/explodedf.csv')

In [222]:
len(megalist2)

4120

In [260]:
df.iloc[0]['reviews'][0]

'Your team in every random competitive game:\n\n- You\n- Russian Guy\n- Another Russian Guy\n- A 6 year old kid who slept with your mother\n- Russian Guy who speaks English\n\n10/10'

In [224]:
megalist3 = []

for rows in df.reviews:
    for items in rows:
        megalist3.append(items)

In [225]:
len(megalist3)

4120

In [227]:
df.iloc[0]['reviews'][0]

'Your team in every random competitive game:\n\n- You\n- Russian Guy\n- Another Russian Guy\n- A 6 year old kid who slept with your mother\n- Russian Guy who speaks English\n\n10/10'

# Beginning Topic Modeling

the df did not save the same way, finishing modeling in here

In [237]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

## Starting with countvectorizer first

In [246]:
cv1 = CountVectorizer(stop_words='english', token_pattern='[a-zA-Z]+', min_df=4)
doc_word = cv1.fit_transform(megalist)

In [247]:
doc_word.shape

(4120, 4990)

In [249]:
#pd.DataFrame(doc_word.toarray(), columns=cv1.get_feature_names()).head(10)

In [250]:
#initializing lsa and nmf that can be reused
lsa = TruncatedSVD(5)
nmf_model = NMF(5)

In [251]:
doc_topic = lsa.fit_transform(doc_word)

In [253]:
doc_topic

array([[ 7.32788665e-01, -3.02413659e-01, -2.25442946e-01,
        -1.22148484e-01, -3.18414335e-03],
       [ 5.65305821e-02, -2.46277401e-02, -1.46856057e-02,
        -7.24282228e-03, -1.09986023e-04],
       [ 1.68311393e-03, -4.91735458e-04, -6.42723232e-04,
        -5.02004997e-04, -3.77152622e-05],
       ...,
       [ 1.56137462e-01, -5.97353695e-02, -4.22314522e-02,
        -9.75426299e-03, -1.11893845e-03],
       [ 1.13960463e+00, -3.96189901e-01, -3.23354180e-01,
        -2.02167376e-01, -4.42368670e-03],
       [ 1.27202736e+00, -3.45710894e-01, -2.94418677e-01,
        -1.85168186e-01, -7.59411026e-03]])